problem

the data is bad, it has duplicate entries with tx hash and time

idea: find how often this occurs

if its not too many delete the entries 

or clean the data and change the times

In [1]:
import pandas as pd
from datetime import datetime

In [55]:
df_full = pd.read_csv("/Users/haaroony/OneDrive - Pometry Ltd/nft_andrea/Data_API.csv")

/var/folders/rm/nnz28pc96cv2xqzbwlwvwl2r0000gn/T/ipykernel_26651/1900226705.py:1: DtypeWarning: Columns (0,1,2,4,6,8,9,10,14,15,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_full = pd.read_csv("/Users/haaroony/OneDrive - Pometry Ltd/nft_andrea/Data_API.csv")


In [49]:
drop_some_column = ["Unique_id_collection", "Permanent_link", "Description", "Name", "Image_url_4", "Image_url_3", "Image_url_2", "Image_url_1"]
df_dropped = df_full.drop(columns=drop_some_column)

In [3]:
df = df_full[['ID_token', 'Transaction_hash', 'Datetime_updated_seconds']]

In [5]:
s = "2019-11-30 23:59:33"
df['unix_time'] = df['Datetime_updated_seconds'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S").timestamp())

/var/folders/rm/nnz28pc96cv2xqzbwlwvwl2r0000gn/T/ipykernel_26651/3612290062.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['unix_time'] = df['Datetime_updated_seconds'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S").timestamp())


old=(1540436348,0xfbf193668516edb858e1ac8b95a562d2a8583418e9dd3cf523b66e6a7426e17c),
new=(1540436348,0x3cd1d9caa9d06e40aef1b052676c72b66d9de81d68dc0d0a0576fc0078a8dabd)

In [31]:
from collections import Counter

print(" NFTS: \n" + str(len(set(df['ID_token']))))

c=0
bad_nfts = set([])
for tid, gp_df in df.groupby(by=['ID_token']):
    c+=1
    if c % 200_000 == 0:
        print(c)
    pairs = list(zip(gp_df['Transaction_hash'],gp_df['unix_time']))
    counted = Counter(pairs)
    counted_total = set(counted.values())
    if max(counted_total) >= 2:
        print("Found a bad nft " + str(tid))
        bad_nfts.add(tid)

 NFTS: 
4746857
200000
400000
600000
800000
1000000
1200000
1400000
1600000
1800000
2000000
2200000
2400000
2600000
2800000
3000000
3200000
3400000
3600000
3800000
4000000
4200000
4400000
Found a bad nft 520975
Found a bad nft 597444
Found a bad nft 614950
Found a bad nft 616766
Found a bad nft 630892
Found a bad nft 640086
Found a bad nft 6877
Found a bad nft 712882
Found a bad nft 712913
4600000
Found a bad nft 856183
Found a bad nft 9103
Found a bad nft 9672
Found a bad nft 992744


java.lang.Error: Mismatched duplicate element: old=(1540436348,0xfbf193668516edb858e1ac8b95a562d2a8583418e9dd3cf523b66e6a7426e17c), new=(1540436348,0x3cd1d9caa9d06e40aef1b052676c72b66d9de81d68dc0d0a0576fc0078a8dabd
d68dc0d0a0576fc0078a8dabd)
        at com.raphtory.internals.storage.pojograph.OrderedBuffer$OrderedAppend.sortedAppend(OrderedBuffer.scala:34)
        at com.raphtory.internals.storage.pojograph.entities.internal.MutableProperty.update(MutableProperty.scala:27)
        at com.raphtory.internals.storage.pojograph.entities.internal.PojoEntity.$plus(PojoEntity.scala:57)
        at com.raphtory.internals.storage.pojograph.PojoBasedPartition.$anonfun$addProperties$1(PojoBasedPartition.scala:33)
        at com.raphtory.internals.storage.pojograph.PojoBasedPartition.$anonfun$addProperties$1$adapted(PojoBasedPartition.scala:32)
        at scala.collection.IterableOnceOps.foreach(IterableOnce.scala:563)
        at scala.collection.IterableOnceOps.foreach$(IterableOnce.scala:561)
        at scala.collection.AbstractIterable.foreach(Iterable.scala:926)
        at com.raphtory.internals.storage.pojograph.PojoBasedPartition.addProperties(PojoBasedPartition.scala:32)
        at com.raphtory.internals.storage.pojograph.PojoBasedPartition.addEdge(PojoBasedPartition.scala:231)
        at com.raphtory.internals.components.partition.BatchWriter.processEdgeAdd(BatchWriter.scala:58)
        at com.raphtory.internals.components.partition.BatchWriter.handleMessage(BatchWriter.scala:29)
        at com.raphtory.api.input.GraphBuilder.handleEdgeAdd(GraphBuilder.scala:267)
        at com.raphtory.api.input.GraphBuilder.addEdge(GraphBuilder.scala:239)
        at com.raphtory.api.input.GraphBuilder.addEdge$(GraphBuilder.scala:231)
        at com.raphtory.examples.nft.graphbuilder.NFTGraphBuilder.addEdge(NFTGraphBuilder.scala:12)
        at com.raphtory.examples.nft.graphbuilder.NFTGraphBuilder.parseTuple(NFTGraphBuilder.scala:96)
        at com.raphtory.examples.nft.graphbuilder.NFTGraphBuilder.parseTuple(NFTGraphBuilder.scala:12)
        at com.raphtory.internals.components.partition.LocalBatchHandler.$anonfun$runIngestion$1(LocalBatchHandler.scala:54)

In [35]:
len(bad_nfts)

13

In [36]:
len(bad_nfts)/4746857*100

0.00027386542295249256

In [52]:
df_dropped.to_csv("/Users/haaroony/OneDrive - Pometry Ltd/nft_andrea/Data_API_cleaned.csv", index=False)

In [53]:
len(df_dropped)

6071027

In [63]:
df_dropped

Smart_contract              5109417
ID_token                    6071027
Transaction_hash            2885387
Seller_address              6071027
Seller_username             1093254
Buyer_address               6071027
Buyer_username               892863
Price_Crypto                6071027
Crypto                      6071027
Price_USD                   6062744
Collection                  6071027
Market                      6071027
Datetime_updated            6071027
Datetime_updated_seconds    6071027
Collection_cleaned          6071027
Category                    6071027
dtype: int64

In [65]:
set(df_dropped['Crypto'])

{'0xBTC',
 '1MT',
 '2XDN',
 'ABST',
 'AMPL',
 'ANRX',
 'ARCONA',
 'ART',
 'ASLT',
 'ATRI',
 'AVRT',
 'B0T',
 'BAEPAY',
 'BASED',
 'BAT',
 'BLVD',
 'BON',
 'BONDLY',
 'BONES',
 'BOOB',
 'BORG',
 'BPC',
 'BUDGET',
 'BUDZ',
 'BZN',
 'CAMEL',
 'CGG',
 'CHERRY',
 'CHONK',
 'COIN',
 'COVAL',
 'CPT',
 'CRED',
 'CUBE',
 'CURIO',
 'DAI',
 'DAPPT',
 'DDIM',
 'DENA',
 'DGX',
 'DHC',
 'DOOM',
 'DUST',
 'EBB',
 'ECAT',
 'ECTO',
 'EGGS',
 'ELAND',
 'ELET',
 'EMONT',
 'ENJ',
 'EPIC',
 'ETH',
 'FIRST',
 'FRFY',
 'FTHR',
 'FTM',
 'FUD',
 'GALA',
 'GCASH',
 'GEM',
 'GMX',
 'GOKU',
 'GOOSE',
 'GOU',
 'GPL',
 'GUSD',
 'HIVED',
 'HOUR',
 'HUE',
 'IMP',
 'INI',
 'INK',
 'JBG',
 'JGN',
 'KAP',
 'KEK',
 'KEK-DEP',
 'KING',
 'KIWI',
 'KLTR',
 'KOI',
 'LAR',
 'LESS',
 'LINK',
 'LIT',
 'LOAD',
 'MANA',
 'MATIC',
 'MBC',
 'MCX',
 'MEME',
 'MGDv2',
 'MKR',
 'MM',
 'MNFT',
 'MORK',
 'MX',
 'NDR',
 'NEWS',
 'NUGS',
 'NVT',
 'OLDWAXIE',
 'OLDWZX',
 'PGU',
 'PIXEL',
 'PIXIE',
 'PLAY',
 'PMON',
 'PPDEX',
 'PRIME',
 'PX

In [66]:
df_dropped_eth_only = df_dropped[df_dropped['Crypto']=='ETH']

In [68]:
df_dropped_eth_only.to_csv("/Users/haaroony/OneDrive - Pometry Ltd/nft_andrea/Data_API_cleaned_ETH_only.csv", index=False)

In [69]:
max(df_dropped_eth_only['Datetime_updated_seconds'])

'2021-04-27 23:59:51'

In [60]:
for i, x in enumerate(df_dropped.columns):
    print(i, x)

0 Smart_contract
1 ID_token
2 Transaction_hash
3 Seller_address
4 Seller_username
5 Buyer_address
6 Buyer_username
7 Price_Crypto
8 Crypto
9 Price_USD
10 Collection
11 Market
12 Datetime_updated
13 Datetime_updated_seconds
14 Collection_cleaned
15 Category


In [50]:
max)df_clean_dropped_nfts)

NameError: name 'df_clean_dropped_nfts' is not defined

## unclean data

In [43]:
unclean_df = pd.read_csv("/Users/haaroony/OneDrive - Pometry Ltd/nft_andrea/bad_nft.csv")

In [44]:
unclean_df

,Smart_contract,ID_token,Transaction_hash,Seller_address,Seller_username,Buyer_address,Buyer_username,Image_url_1,Image_url_2,Image_url_3,...,Name,Description,Collection,Market,Datetime_updated,Datetime_updated_seconds,Permanent_link,Unique_id_collection,Collection_cleaned,Category
0,0x06012c8cf97bead5deae237070f9587f8e7a266d,539565,0xb0dd3d7199c3df1e81f99f304a1bfc834373684468aa...,0x4886404ecfb86b759b27867afb0f64172eea1c53,PsyDuck,0xefe92c9018c8fd8bf52a2107f6177078a291353f,NaN,https://storage.opensea.io/0x06012c8cf97bead5d...,https://img.cryptokitties.co/0x06012c8cf97bead...,https://storage.opensea.io/0x06012c8cf97bead5d...,...,💜💙 Wuvme Chronic Bobtail,Bonjour! I'm 💜💙 Wuvme Chronic Bobtail. I work ...,Cryptokitties,OpenSea,2018-02-24 00:00:00,2018-02-24 20:21:59,https://opensea.io/assets/0x06012c8cf97bead5de...,"('Cryptokitties', '539565')",Cryptokittie,Art


In [47]:
unclean_df.drop(columns=drop_some_column)

,Smart_contract,ID_token,Transaction_hash,Seller_address,Seller_username,Buyer_address,Buyer_username,Price_Crypto,Crypto,Price_USD,Collection,Market,Datetime_updated,Datetime_updated_seconds,Collection_cleaned,Category
0,0x06012c8cf97bead5deae237070f9587f8e7a266d,539565,0xb0dd3d7199c3df1e81f99f304a1bfc834373684468aa...,0x4886404ecfb86b759b27867afb0f64172eea1c53,PsyDuck,0xefe92c9018c8fd8bf52a2107f6177078a291353f,NaN,0.004,ETH,3.37072,Cryptokitties,OpenSea,2018-02-24 00:00:00,2018-02-24 20:21:59,Cryptokittie,Art


In [71]:
df_dropped_eth_only[df_dropped_eth_only['Price_USD']==16.30137838541667]

,Smart_contract,ID_token,Transaction_hash,Seller_address,Seller_username,Buyer_address,Buyer_username,Price_Crypto,Crypto,Price_USD,Collection,Market,Datetime_updated,Datetime_updated_seconds,Collection_cleaned,Category
4173556,0x06012c8cf97bead5deae237070f9587f8e7a266d,640086,0xd8d8a2c747dda3783513f95bfa9bd6935cee505d873a...,0xfb9e56d1883b2c7d6c4ca544b20a647d5688ba27,NaN,0xbca1d1ee93827d26bd3e5ccb4e193c792489a172,Waltmester3000,0.031068,ETH,16.301378,Cryptokitties,OpenSea,2018-03-25 00:00:00,2018-03-25 08:12:18,Cryptokittie,Art


,640086,0xd8d8a2c747dda3783513f95bfa9bd6935cee505d873aa1036024eb178750627d,0xfb9e56d1883b2c7d6c4ca544b20a647d5688ba27,,0xbca1d1ee93827d26bd3e5ccb4e193c792489a172,,0.0310679687499999,ETH,16.301363203125,Cryptokitties,Cryptokitties,2018-03-24 00:00:00,2018-03-24 13:38:15,Cryptokittie,Art
0x06012c8cf97bead5deae237070f9587f8e7a266d,640086,0xd8d8a2c747dda3783513f95bfa9bd6935cee505d873aa1036024eb178750627d,0xfb9e56d1883b2c7d6c4ca544b20a647d5688ba27,,0xbca1d1ee93827d26bd3e5ccb4e193c792489a172,Waltmester3000,0.0310679976851851,ETH,16.30137838541667,Cryptokitties,OpenSea,2018-03-25 00:00:00,2018-03-25 08:12:18,Cryptokittie,Art
0x06012c8cf97bead5deae237070f9587f8e7a266d,640086,0xd8d8a2c747dda3783513f95bfa9bd6935cee505d873aa1036024eb178750627d,0xfb9e56d1883b2c7d6c4ca544b20a647d5688ba27,,0xbca1d1ee93827d26bd3e5ccb4e193c792489a172,Waltmester3000,0.0310679687499999,ETH,16.301363203125,Cryptokitties,OpenSea,2018-03-25 00:00:00,2018-03-25 08:12:18,Cryptokittie,Art

#### How many rows are there?

In [81]:
len(df_dropped_eth_only)

2405225

#### How many txs are there unique

In [77]:
len(df_dropped_eth_only['Transaction_hash'].unique())

2053368

#### how many tx entries are duplicated

In [85]:
counted_hashes = Counter(list(df_dropped_eth_only['Transaction_hash']))

In [115]:
len([ x[0] for x in counted_hashes.items() if x[1]>1 ])

350447

### How many nfts are there?

In [91]:
len())

1641490

In [117]:
counted_hashes['0x78aa25162224b01c9f46c00a77fa0c00e9b62003a72425c60ed486ac5ccccf5b']

1

### How many nfts are duplicated?

In [95]:
dupped_hashes = set([ x[0] for x in counted_hashes.items() if x[1]>1 ])

In [98]:
nfts_with_dups = set(df_dropped_eth_only[df_dropped_eth_only['Transaction_hash'].isin(dupped_hashes)]['ID_token'])

In [99]:
len(nfts_with_dups)

638431

In [101]:
clean_nfts = set(df_dropped_eth_only['ID_token'])-(nfts_with_dups)

In [102]:
df_with_clean_nfs = df_dropped_eth_only[df_dropped_eth_only['ID_token'].isin(clean_nfts)]

In [103]:
len(df_with_clean_nfs)

1617637

In [105]:
df_with_clean_nfs.to_csv("/Users/haaroony/OneDrive - Pometry Ltd/nft_andrea/Data_API_clean_nfts_ETH_only.csv", index=False)

In [106]:
len(df_with_clean_nfs)

1617637

In [108]:
max(df_with_clean_nfs['Datetime_updated_seconds'])

'2021-04-27 23:59:51'

In [114]:
228 in clean_nfts

False

### Clean data with timestamp, buyer address, token_id 

In [135]:
df_dropped_eth_only['comb_label'] =df_dropped_eth_only.apply (lambda row: str(row['Buyer_address'])+'_'+str(row['ID_token'])+'_'+str(row['Datetime_updated_seconds']), axis=1)

/var/folders/rm/nnz28pc96cv2xqzbwlwvwl2r0000gn/T/ipykernel_26651/1258917884.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dropped_eth_only['comb_label'] =df_dropped_eth_only.apply (lambda row: str(row['Buyer_address'])+'_'+str(row['ID_token'])+'_'+str(row['Datetime_updated_seconds']), axis=1)


In [136]:
bad_columns = Counter(list(df_dropped_eth_only['comb_label']))

In [137]:
len(set([ x for x in bad_columns.items() if x[1]>1 ]))

216

In [ ]:
set([ x for x in bad_columns.items() if x[1]>1 ])

#### How many nfts is this?

In [180]:
df_full

,Smart_contract,ID_token,Transaction_hash,Seller_address,Seller_username,Buyer_address,Buyer_username,Image_url_1,Image_url_2,Image_url_3,...,Name,Description,Collection,Market,Datetime_updated,Datetime_updated_seconds,Permanent_link,Unique_id_collection,Collection_cleaned,Category
0,NaN,87094722.0,0xed9ff1c311a611e5c457ce0a49bbbb7ed55efca98842...,0x76481caa104b5f6bccb540dae4cefaf1c398ebea,NaN,0xe0fb7622091e3d9ef9b438471b10b9ea88c7cf6b,NaN,https://card.godsunchained.com/?id=33&q=4,NaN,NaN,...,NaN,NaN,Godsunchained,Godsunchained,2019-11-30 00:00:00,2019-11-30 23:59:33,NaN,"('Godsunchained', '87094722.0')",Godsunchained,Games
1,NaN,73044643.0,0xbc8412594ffa7382e8306215ece5b4e041fc641b1813...,0x76481caa104b5f6bccb540dae4cefaf1c398ebea,NaN,0x2c00d05ff7ffe93d77145dfe12aa56e622ae6f40,NaN,https://card.godsunchained.com/?id=79&q=4,NaN,NaN,...,NaN,NaN,Godsunchained,Godsunchained,2019-11-30 00:00:00,2019-11-30 23:59:33,NaN,"('Godsunchained', '73044643.0')",Godsunchained,Games
2,NaN,17107482.0,0x92672baf5e9e23de230dee8e57ddf443ab6d5a336341...,0x76481caa104b5f6bccb540dae4cefaf1c398ebea,NaN,0xe0fb7622091e3d9ef9b438471b10b9ea88c7cf6b,NaN,https://card.godsunchained.com/?id=33&q=4,NaN,NaN,...,NaN,NaN,Godsunchained,Godsunchained,2019-11-30 00:00:00,2019-11-30 23:59:33,NaN,"('Godsunchained', '17107482.0')",Godsunchained,Games
3,NaN,82230857.0,0x3d157249e3083622c2d756f07a05caa3a76687288b32...,0x76481caa104b5f6bccb540dae4cefaf1c398ebea,NaN,0x2c00d05ff7ffe93d77145dfe12aa56e622ae6f40,NaN,https://card.godsunchained.com/?id=227&q=4,NaN,NaN,...,NaN,NaN,Godsunchained,Godsunchained,2019-11-30 00:00:00,2019-11-30 23:59:33,NaN,"('Godsunchained', '82230857.0')",Godsunchained,Games
4,NaN,40925443.0,0xa892b3569dc8e6b3c7db93aade62372637ebbc61fb63...,0x76481caa104b5f6bccb540dae4cefaf1c398ebea,NaN,0x2c00d05ff7ffe93d77145dfe12aa56e622ae6f40,NaN,https://card.godsunchained.com/?id=62&q=4,NaN,NaN,...,NaN,NaN,Godsunchained,Godsunchained,2019-11-30 00:00:00,2019-11-30 23:58:44,NaN,"('Godsunchained', '40925443.0')",Godsunchained,Games
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6071022,0xb932a70a57673d89f4acffbe830e8ed7f75fb9e0,16412,0x346561b3f7a233c4245f91c68940b2d750df26ef1b04...,0x9f37284b3a538be8eea2d401b595f44c96d39b70,NaN,0xa353401738e7ec6a4db9c0ee4c66eb5252f681c4,NaN,https://lh3.googleusercontent.com/isSUPEsJJWpF...,https://ipfs.pixura.io/ipfs/QmSHxximuHUZV54pAA...,https://lh3.googleusercontent.com/isSUPEsJJWpF...,...,Pájaro Doctor 13/40,Digital piece made by using a variety of softw...,Superrare,OpenSea,2021-04-01 00:00:00,2021-04-01 00:00:52,https://opensea.io/assets/0xb932a70a57673d89f4...,"('Superrare', '16412')",Superrare,Art
6071023,0x495f947276749ce646f68ac8c248420045cb7b5e,1062274196422536302774639391540663457511117433...,0xd507715bb835252a65d572037ac8eb36854b05ecc672...,0xeada96bf164643bdfe3e657a9f380854694a49dd,Mistcop,0xe2fe53f9bcb33e3e0fc65f2f627ca4eb8e958347,NaN,https://lh3.googleusercontent.com/qdzZDtYHHVdi...,NaN,https://lh3.googleusercontent.com/qdzZDtYHHVdi...,...,Kawaii Monster #2,NaN,Kawaii-monsters,OpenSea,2021-04-01 00:00:00,2021-04-01 00:00:50,https://opensea.io/assets/0x495f947276749ce646...,"('Kawaii-monsters', '1062274196422536302774639...",Kawaiimonster,Other
6071024,0x495f947276749ce646f68ac8c248420045cb7b5e,4602588071276327231659212203812866312085457052...,0x4cb90a775619a9483c003050de47c403fae957e5352b...,0x65c1b9ae4e4d8dcccfd3dc41b940840fe8570f2a,International_Megadigital,0x220d20310f52d97e353028ebd742d2c690eaf0a1,WanderlustNFT,https://lh3.googleusercontent.com/SoTAV8k4UC90...,NaN,https://lh3.googleusercontent.com/SoTAV8k4UC90...,...,Bishoujo Originals - #4977,The original source for collectible Bishoujo h...,Moe-tokens,OpenSea,2021-04-01 00:00:00,2021-04-01 00:00:51,https://opensea.io/assets/0x495f947276749ce646...,"('Moe-tokens', '460258807127632723165921220381...",Moetoken,Other
6071025,0x495f947276749ce646f68ac8c248420045cb7b5e,8060875342503467325201444233539848397816312727...,0x76068

In [139]:
bad_buyers= set([x.split('_')[0] for x in bad_columns])

In [140]:
bad_nfts = set(df_dropped_eth_only[df_dropped_eth_only['Buyer_address'].isin(bad_buyers)]['ID_token'])

In [144]:
len(bad_nfts), len(set(df_dropped_eth_only['ID_token']))

(1641490, 1641490)

In [141]:
df_no_bad_nfts = df_dropped_eth_only[~df_dropped_eth_only['ID_token'].isin(bad_nfts)]

In [133]:
df_no_bad_nfts.to_csv("/Users/haaroony/OneDrive - Pometry Ltd/nft_andrea/Data_API_clean_nfts_ETH_only.csv", index=False)

In [134]:
max(df_no_bad_nfts['Datetime_updated_seconds'])

'2021-04-27 23:59:51'

In [147]:
x = df_dropped_eth_only.groupby(['ID_token']).count().reset_index()

In [152]:
x.max()

/var/folders/rm/nnz28pc96cv2xqzbwlwvwl2r0000gn/T/ipykernel_26651/557561640.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  x.max()


Smart_contract              7524
Transaction_hash            7524
Seller_address              7524
Seller_username             7292
Buyer_address               7524
Buyer_username              6041
Price_Crypto                7524
Crypto                      7524
Price_USD                   7524
Collection                  7524
Market                      7524
Datetime_updated            7524
Datetime_updated_seconds    7524
Collection_cleaned          7524
Category                    7524
comb_label                  7524
dtype: int64

In [168]:
nfts_with_double = set()
allowd_cols = ['Transaction_hash', 'Seller_address', 'Buyer_address', 'Price_Crypto','Crypto', 'Price_USD', 'Market', 'Datetime_updated_seconds']
for i, row in x.iterrows():
    if i % 100_000 == 0:
        print("%i/%i - bad found %i" % (i, len(x), len(nfts_with_double)))
    max_found = max([row.Transaction_hash, row.Crypto, row.Price_USD, row.Market, row.Datetime_updated_seconds, row.Buyer_address])
    if max_found > 1:
        nfts_with_double.add(row.ID_token)

0/1641490 - bad found 0
100000/1641490 - bad found 30437
200000/1641490 - bad found 50412
300000/1641490 - bad found 71674
400000/1641490 - bad found 86999
500000/1641490 - bad found 97145
600000/1641490 - bad found 105076
700000/1641490 - bad found 111792
800000/1641490 - bad found 118677
900000/1641490 - bad found 140370
1000000/1641490 - bad found 158492
1100000/1641490 - bad found 181904
1200000/1641490 - bad found 213500
1300000/1641490 - bad found 250414
1400000/1641490 - bad found 283129
1500000/1641490 - bad found 311430
1600000/1641490 - bad found 340325


In [177]:
640086 in nfts_with_double 

False

In [169]:
df_dropped_eth_only_no_double_entities = df_dropped_eth_only[~df_dropped_eth_only['ID_token'].isin(nfts_with_double)]

In [170]:
len(df_dropped_eth_only_no_double_entities)

1290107

In [171]:
df_dropped_eth_only_no_double_entities.to_csv("/Users/haaroony/OneDrive - Pometry Ltd/nft_andrea/Data_API_clean_nfts_ETH_only.csv", index=False)

In [173]:
max(df_dropped_eth_only_no_double_entities['Datetime_updated_seconds'])

'2021-04-27 23:59:51'

In [179]:
df_dropped_eth_only_no_double_entities[df_dropped_eth_only_no_double_entities['ID_token']==640086]

,Smart_contract,ID_token,Transaction_hash,Seller_address,Seller_username,Buyer_address,Buyer_username,Price_Crypto,Crypto,Price_USD,Collection,Market,Datetime_updated,Datetime_updated_seconds,Collection_cleaned,Category,comb_label
562199,NaN,640086,0xd8d8a2c747dda3783513f95bfa9bd6935cee505d873a...,0xfb9e56d1883b2c7d6c4ca544b20a647d5688ba27,NaN,0xbca1d1ee93827d26bd3e5ccb4e193c792489a172,NaN,0.031068,ETH,16.301363,Cryptokitties,Cryptokitties,2018-03-24 00:00:00,2018-03-24 13:38:15,Cryptokittie,Art,0xbca1d1ee93827d26bd3e5ccb4e193c792489a172_640...


## Reclean data

In [ ]:
df_full = pd.read_csv("/Users/haaroony/OneDrive - Pometry Ltd/nft_andrea/Data_API.csv")
drop_some_column = ["Unique_id_collection", "Permanent_link", "Description", "Name", "Image_url_4", "Image_url_3", "Image_url_2", "Image_url_1"]
df_dropped = df_full.drop(columns=drop_some_column)

In [203]:
df_dropped['ID_token'] = df_dropped['ID_token'].astype(str)
df_dropped['Transaction_hash'] = df_dropped['Transaction_hash'].astype(str)
df_dropped['Seller_address'] = df_dropped['Seller_address'].astype(str)
df_dropped['Buyer_address'] = df_dropped['Buyer_address'].astype(str)
df_dropped['Crypto'] = df_dropped['Crypto'].astype(str)
df_dropped['Market'] = df_dropped['Market'].astype(str)
df_dropped['Datetime_updated_seconds'] = df_dropped['Datetime_updated_seconds'].astype(str)

In [204]:
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6071027 entries, 0 to 6071026
Data columns (total 16 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   Smart_contract            object 
 1   ID_token                  object 
 2   Transaction_hash          object 
 3   Seller_address            object 
 4   Seller_username           object 
 5   Buyer_address             object 
 6   Buyer_username            object 
 7   Price_Crypto              float64
 8   Crypto                    object 
 9   Price_USD                 float64
 10  Collection                object 
 11  Market                    object 
 12  Datetime_updated          object 
 13  Datetime_updated_seconds  object 
 14  Collection_cleaned        object 
 15  Category                  object 
dtypes: float64(2), object(14)
memory usage: 741.1+ MB


In [205]:
df_eth = df_dropped[df_dropped['Crypto']=='ETH'] 

In [217]:
len(df_eth)

2405225

In [ ]:
df_eth.drop_duplicates(inplace=True)

In [219]:
len(df_eth)

2405225

In [222]:
grouped

,Smart_contract,ID_token,Transaction_hash,Seller_address,Seller_username,Buyer_address,Buyer_username,Price_Crypto,Crypto,Price_USD,Collection,Market,Datetime_updated,Datetime_updated_seconds,Collection_cleaned,Category
4169375,0xbb5ed1edeb5149af3ab43ea9c7a6963b3c1374f7,0,0xd7d21d15a331fe2445f1d1720236a35fb431ec706a1a...,0x53ec525b9df849be99263311f5b7f9fd6cfebea6,flamey,0xe820f08968489c466c77c2ea1a5c436d8c70d896,NaN,20.065799,ETH,14917.917977,Cryptocelebrities,OpenSea,2018-02-05 00:00:00,2018-02-05 04:10:37,Cryptocelebritie,Collectible
4257282,0x5d0ee539893323f9965e3ac2846ea6421255ae79,0,0xfaef63e4429b198ab643c33e6f2674588425abe391d8...,0xd04c453461a77b342fea9d1fec55861399401122,OniOni,0x183febd8828a9ac6c70c0e27fbf441b93004fc05,bjork,0.200000,ETH,76.530000,Hiprecious,OpenSea,2018-08-07 00:00:00,2018-08-07 00:39:41,Hiprecio,Other
4317769,0x079b18d6556e198c98e599490c6cbba231ab4db8,0,0x077f2e2be7f1b1a84cd545e1ecce65131550685efb29...,0x2c67f4a0434c4d1796d443124ebc1ac1d8790ebb,SKIBITSKY,0xa0040d7049ed4b2aa56ff785b77a5e305be098b9,NaN,0.600000,ETH,65.592000,Crypto-bundle-sale,OpenSea,2018-12-20 00:00:00,2018-12-20 21:49:40,Cryptobundlesale,Other
4362406,0xcd65bacacea37e65eac019f37f5cb38ae62a7fbf,0,0x87c4941dcf9d860033f7a286f35630526bca98375725...,0x1e55c85801a2c4f0bec57c84742a8ef3d72de57b,0x1e55c85801a2c4f0bec57c84742a8ef3d72de57b,0x07c622e3a3c9eb73f02effee4cfc189897bff467,NaN,0.050000,ETH,6.789500,Krystal-kart-ar-vehicles-factory,OpenSea,2019-02-28 00:00:00,2019-02-28 22:07:45,Krystalkartarvehiclesfactory,Other
4362502,0xcd65bacacea37e65eac019f37f5cb38ae62a7fbf,0,0xa6d1486b4192dc225280e6b902d8ea4960ac27ead108...,0x1e55c85801a2c4f0bec57c84742a8ef3d72de57b,0x1e55c85801a2c4f0bec57c84742a8ef3d72de57b,0x2cb86d919332d0369c66a2d5982419266f5e490f,OpenSeaDeals,0.050000,ETH,6.789500,Krystal-kart-ar-vehicles-factory,OpenSea,2019-02-28 00:00:00,2019-02-28 19:37:17,Krystalkartarvehiclesfactory,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6068466,0xdf801468a808a32656d2ed2d2d80b72a129739f4,0,0x718c87d6e86b158dce613c65eb832bb11ea104c3497c...,0x8f06ed205df2041233b2aa022e945af9c230c49b,Jelenowski,0x690b6403e751dc5d4da66698eebe45567a600946,NaN,0.035000,ETH,67.739525,Somnium-space,OpenSea,2021-04-01 00:00:00,2021-04-01 13:48:42,Somniumspace,Metaverse
6069704,0xd2d2a84f0eb587f70e181a0c4b252c2c053f80cb,0,0xf491e07639e940e3e12fdcd65bfb513769bd21d993c9...,0x97575aac6912233403e9b8935e980dec40c55548,Flayed,0xd79791d259520a9765953be41f680c8bc05e8dbc,MARLLO,0.059000,ETH,114.189485,Toshimon-minter,OpenSea,2021-04-01 00:00:00,2021-04-01 07:08:43,Toshimonminter,Games
6069988,0xd2d2a84f0eb587f70e181a0c4b252c2c053f80cb,0,0x1820f6eb3a8c93d5c1eb1d7e9b0c5a8b9fa448427976...,0xcbf3f36a83b80c546c7e56abd7fbb547946b6ed4,Korakis,0x745fde0961988a345f5348fde6d4e7bf38396f23,risiNFT,0.040000,ETH,77.416600,Toshimon-minter,OpenSea,2021-04-01 00:00:00,2021-04-01 06:05:54,Toshimonminter,Games
6070287,0xdf801468a808a32656d2ed2d2d80b72a129739f4,0,0x4809fa181e17e63b7bd90defba624cf359703af13c8d...,0x8f06ed205df2041233b2aa022e945af9c230c49b,Jelenowski,0x8ddf7b6da2c482ed6b2d3b793cacdf22c65396d6,Corie,0.030000,ETH,58.062450,Somnium-space,OpenSea,2021-04-01 00:00:00,2021-04-01 04:03:20,Somniumspace,Metaverse


In [239]:
nfts_with_double_time = set()
allowd_cols = ['Transaction_hash', 'Seller_address', 'Buyer_address', 'Price_Crypto','Crypto', 'Price_USD', 'Market', 'Datetime_updated_seconds']
print("Doing groupby")
i=0
total = len(set(df_eth['ID_token']))
i=0
for j, grouped in df_eth.groupby(['ID_token']):
    i+=1
    if i % 100_000 == 0:
        print("%i/%i - bad found %i" % (i, total, len(nfts_with_double_time)))
    all_dates = list(grouped['Datetime_updated_seconds'])
    setted = set(all_dates)
    if len(all_dates) != len(setted):
        nfts_with_double_time.add(grouped.iloc[0].ID_token)

Doing groupby
100000/1235989 - bad found 25
200000/1235989 - bad found 43
300000/1235989 - bad found 64
400000/1235989 - bad found 87
500000/1235989 - bad found 106
600000/1235989 - bad found 142
700000/1235989 - bad found 210
800000/1235989 - bad found 260
900000/1235989 - bad found 304
1000000/1235989 - bad found 337
1100000/1235989 - bad found 381
1200000/1235989 - bad found 463


In [241]:
df_eth_singles = df_eth[~df_eth['ID_token'].isin(nfts_with_double_time)]

In [242]:
df_eth_singles.to_csv("/Users/haaroony/OneDrive - Pometry Ltd/nft_andrea/Data_API_clean_nfts_ETH_only.csv", index=False)

In [244]:
max(df_eth_singles['Datetime_updated_seconds'])

'2021-04-27 23:59:51'

In [243]:
df_eth_singles

,Smart_contract,ID_token,Transaction_hash,Seller_address,Seller_username,Buyer_address,Buyer_username,Price_Crypto,Crypto,Price_USD,Collection,Market,Datetime_updated,Datetime_updated_seconds,Collection_cleaned,Category
0,NaN,87094722.0,0xed9ff1c311a611e5c457ce0a49bbbb7ed55efca98842...,0x76481caa104b5f6bccb540dae4cefaf1c398ebea,NaN,0xe0fb7622091e3d9ef9b438471b10b9ea88c7cf6b,NaN,0.0002,ETH,0.030318,Godsunchained,Godsunchained,2019-11-30 00:00:00,2019-11-30 23:59:33,Godsunchained,Games
1,NaN,73044643.0,0xbc8412594ffa7382e8306215ece5b4e041fc641b1813...,0x76481caa104b5f6bccb540dae4cefaf1c398ebea,NaN,0x2c00d05ff7ffe93d77145dfe12aa56e622ae6f40,NaN,0.0013,ETH,0.197067,Godsunchained,Godsunchained,2019-11-30 00:00:00,2019-11-30 23:59:33,Godsunchained,Games
2,NaN,17107482.0,0x92672baf5e9e23de230dee8e57ddf443ab6d5a336341...,0x76481caa104b5f6bccb540dae4cefaf1c398ebea,NaN,0xe0fb7622091e3d9ef9b438471b10b9ea88c7cf6b,NaN,0.0002,ETH,0.030318,Godsunchained,Godsunchained,2019-11-30 00:00:00,2019-11-30 23:59:33,Godsunchained,Games
3,NaN,82230857.0,0x3d157249e3083622c2d756f07a05caa3a76687288b32...,0x76481caa104b5f6bccb540dae4cefaf1c398ebea,NaN,0x2c00d05ff7ffe93d77145dfe12aa56e622ae6f40,NaN,0.0022,ETH,0.333498,Godsunchained,Godsunchained,2019-11-30 00:00:00,2019-11-30 23:59:33,Godsunchained,Games
4,NaN,40925443.0,0xa892b3569dc8e6b3c7db93aade62372637ebbc61fb63...,0x76481caa104b5f6bccb540dae4cefaf1c398ebea,NaN,0x2c00d05ff7ffe93d77145dfe12aa56e622ae6f40,NaN,0.0012,ETH,0.181908,Godsunchained,Godsunchained,2019-11-30 00:00:00,2019-11-30 23:58:44,Godsunchained,Games
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6071022,0xb932a70a57673d89f4acffbe830e8ed7f75fb9e0,16412,0x346561b3f7a233c4245f91c68940b2d750df26ef1b04...,0x9f37284b3a538be8eea2d401b595f44c96d39b70,NaN,0xa353401738e7ec6a4db9c0ee4c66eb5252f681c4,NaN,0.5000,ETH,967.707500,Superrare,OpenSea,2021-04-01 00:00:00,2021-04-01 00:00:52,Superrare,Art
6071023,0x495f947276749ce646f68ac8c248420045cb7b5e,1062274196422536302774639391540663457511117433...,0xd507715bb835252a65d572037ac8eb36854b05ecc672...,0xeada96bf164643bdfe3e657a9f380854694a49dd,Mistcop,0xe2fe53f9bcb33e3e0fc65f2f627ca4eb8e958347,NaN,0.0500,ETH,96.770750,Kawaii-monsters,OpenSea,2021-04-01 00:00:00,2021-04-01 00:00:50,Kawaiimonster,Other
6071024,0x495f947276749ce646f68ac8c248420045cb7b5e,4602588071276327231659212203812866312085457052...,0x4cb90a775619a9483c003050de47c403fae957e5352b...,0x65c1b9ae4e4d8dcccfd3dc41b940840fe8570f2a,International_Megadigital,0x220d20310f52d97e353028ebd742d2c690eaf0a1,WanderlustNFT,0.0185,ETH,35.805177,Moe-tokens,OpenSea,2021-04-01 00:00:00,2021-04-01 00:00:51,Moetoken,Other
6071025,0x495f947276749ce646f68ac8c248420045cb7b5e,8060875342503467325201444233539848397816312727...,0x76068e84592753bb1e076c5320c7e3ecfe41e4da8ebe...,0xb236f00808a9f88b3117af95c881966288dfbcbb,TheOfficialGarrett,0xd543a0be0684f0556786586b83f4c9fd16a4dc35,kittylord,0.0100,ETH,19.354150,Not-fun-tokens,OpenSea,2021-04-01 00:00:00,2021-04-01 00:00:40,Notfuntoken,Other


In [246]:
import pandas as pd
df = pd.read_csv("/Users/haaroony/OneDrive - Pometry Ltd/nft_andrea/results/Data_API_clean_nfts_ETH_only.csv")

/var/folders/rm/nnz28pc96cv2xqzbwlwvwl2r0000gn/T/ipykernel_26651/4193332239.py:2: DtypeWarning: Columns (0,1,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/haaroony/OneDrive - Pometry Ltd/nft_andrea/results/Data_API_clean_nfts_ETH_only.csv")


In [259]:
df['ID_token2'] =df['ID_token'].astype(str)+"_"+df['Collection_cleaned']

In [266]:
df['ID_token'] = df['ID_token2']

In [267]:
df.drop(columns=['ID_token2'], inplace=True)

In [269]:
df.to_csv("/Users/haaroony/OneDrive - Pometry Ltd/nft_andrea/results/Data_API_clean_nfts_ETH_only.csv", index=False)

dd##### 